# **Predicting Electricity Spot Prices Based on Weather Patterns in Nordic Countries**

In this project, I will combine weather, electricity spot price and energy productionn and consumption data for Norway in the perido of 2017-2019.

https://www.statnett.no/en/
https://www.ncdc.noaa.gov/cdo-web/
https://www.energidataservice.dk/tso-electricity/Elspotprices

In [29]:
#Set up and Libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.express as px

print("Libaries imported")

Libaries imported


In [30]:
#Loading Data
weather_df = pd.read_csv('/kaggle/input/finland-norway-and-sweden-weather-data-20152019/nordics_weather.csv')
electricity_df = pd.read_csv('/kaggle/input/electricity-spot-price/Elspotprices.csv', delimiter=';')

# List of production and consumption CSV files.
production_consumption_files = ['/kaggle/input/production-and-consumption-2017-2019/ProductionConsumption-2017.csv', 
                                '/kaggle/input/production-and-consumption-2017-2019/ProductionConsumption-2018.csv', 
                                '/kaggle/input/production-and-consumption-2017-2019/ProductionConsumption-2019.csv']

# Read production and comsumption CSV files into dataframes and concatenate them together.
dfs = [pd.read_csv(file, delimiter=';') for file in production_consumption_files]
production_consumption_df = pd.concat(dfs, ignore_index=True)

print("Datasets Loaded")

Datasets Loaded


In [31]:
# Basic exploration
print(weather_df.head())
print(weather_df.dtypes)
print(weather_df.isnull().sum())

print(electricity_df.head())
print(electricity_df.dtypes)
print(electricity_df.isnull().sum())

print(production_consumption_df.head())
print(production_consumption_df .dtypes)
print(production_consumption_df .isnull().sum())

   country      date  precipitation  snow_depth       tavg      tmax  \
0  Finland  1/1/2015       1.714141  284.545455   1.428571  2.912739   
1  Finland  1/2/2015      10.016667  195.000000   0.553571  2.358599   
2  Finland  1/3/2015       3.956061  284.294118  -1.739286  0.820382   
3  Finland  1/4/2015       0.246193  260.772727  -7.035714 -3.110828   
4  Finland  1/5/2015       0.036364  236.900000 -17.164286 -8.727564   

        tmin  
0  -1.015287  
1  -0.998718  
2  -3.463871  
3  -9.502581  
4 -19.004487  
country           object
date              object
precipitation    float64
snow_depth       float64
tavg             float64
tmax             float64
tmin             float64
dtype: object
country          0
date             0
precipitation    0
snow_depth       0
tavg             0
tmax             0
tmin             0
dtype: int64
            HourUTC            HourDK PriceArea SpotPriceDKK SpotPriceEUR
0  2022-10-19 21:00  2022-10-19 23:00       DK2   978,750000   131,5

Based on the inital exploration of the datasets, we can see there are no missing values but the data structure needs to be cleaned and parsed correctly and the time zones need to be aligned.

Weather Data:
There are no missing values and the data appears to be clean.

The 'date' column is currently type 'object', which needs to be converted to 'datetime' and set as the index. This will allow for easier time series analysis and aslignment with the other datasets.

Electricity Spot Price Data:
Therer are no missing values but the dataset requires some cleaning.

The dataframe formatting means we need to load the data with a delimiter ';'

I will remove the 'SpotPriceDKK' and 'HourDK' columns due to redudency and aligning the time zones from all data sets to UTC.

The 'SpotPriceEUR' column has commas not dots in the decimal place, this will cause issues when convering them to numerical values. These converted to 'float'.

The 'HourUTC'column is a strings, which need to be converted to 'datetime' for time-based analysis just like our other datasets. I will set 'HourUTC' as the index.

Production and Conmsumption Data: ---
The dataframe formatting means we need to load the data with a delimiter ';'
I also need to specify the correct formating for the date and time before converting to datetime and setting it the index.


In [32]:
#Clean Weather Dataset

# Covert 'date' to 'datetime' and set 'date' as index
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df.set_index('date', inplace=True)

print(weather_df.head())
print(weather_df.dtypes)

            country  precipitation  snow_depth       tavg      tmax       tmin
date                                                                          
2015-01-01  Finland       1.714141  284.545455   1.428571  2.912739  -1.015287
2015-01-02  Finland      10.016667  195.000000   0.553571  2.358599  -0.998718
2015-01-03  Finland       3.956061  284.294118  -1.739286  0.820382  -3.463871
2015-01-04  Finland       0.246193  260.772727  -7.035714 -3.110828  -9.502581
2015-01-05  Finland       0.036364  236.900000 -17.164286 -8.727564 -19.004487
country           object
precipitation    float64
snow_depth       float64
tavg             float64
tmax             float64
tmin             float64
dtype: object


In [33]:
#Clean Electricity Spot Price Dataset

# Convert the spot prices to numeric, handling commas and converting to float values.
electricity_df['SpotPriceEUR'] = electricity_df['SpotPriceEUR'].str.replace(',', '').astype(float)

# Convert the 'HourUTC' column to datetime
electricity_df['HourUTC'] = pd.to_datetime(electricity_df['HourUTC'])

# Set 'HourUTC' as index for the electricity data
electricity_df.set_index('HourUTC', inplace=True)

# Drop the redundant columns 'SpotPriceDKK' and 'TimeDKK'
electricity_df = electricity_df.drop(columns=['SpotPriceDKK', 'HourDK'])


print(electricity_df.head())
print(electricity_df.dtypes)

                    PriceArea  SpotPriceEUR
HourUTC                                    
2022-10-19 21:00:00       DK2   131570007.0
2022-10-19 20:00:00       DK2   148149994.0
2022-10-19 19:00:00       DK2   146570007.0
2022-10-19 18:00:00       DK2   166500000.0
2022-10-19 17:00:00       DK2   226919998.0
PriceArea        object
SpotPriceEUR    float64
dtype: object


In [34]:
# Clean Production and Consumption Dataset

# Strip leading/trailing spaces from the 'Time' column
production_consumption_df['Time'] = production_consumption_df['Time'].str.strip()

# Convert 'Time' to datetime without timezone information
production_consumption_df['Time'] = pd.to_datetime(production_consumption_df['Time'], format='%d.%m.%Y %H:%M:%S')

# 3. Set 'Time' as index
production_consumption_df.set_index('Time', inplace=True)

# Display the first few rows and the data types
print(production_consumption_df.head())
print(production_consumption_df.dtypes)


ValueError: unconverted data remains when parsing with format "%d.%m.%Y %H:%M:%S": " +02:00", at position 2018. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

There is an overlaping time frame from 2017 to 2019 which will be where I merge the two data sets for futher analysis.

In [ ]:
# Filter the datasets to match the time range of (2017-2019)
weather_df_filtered = weather_df[
    (weather_df.index >= '2017-01-01') & (weather_df.index <= '2019-12-31')
]
electricity_df_filtered = electricity_df[
    (electricity_df.index >= '2017-01-01') & (electricity_df.index <= '2019-12-31')
]

production_consumption_df_filtered = production_consumption_df[
    (production_consumption_df.index >= '2017-01-01') & (production_consumption_df.index <= '2019-12-31')
]

# Merge the datasets on the 'HourUTC' column (adjust if merging on other columns)
merged_df = pd.merge(electricity_df_filtered, weather_df_filtered, left_on='HourUTC', right_index=True, how='inner')

# Display the first few rows of the merged dataframe
print(merged_df.head())
print(merged_df.dtypes)


In [ ]:
# Remove duplicates (if any)
merged_df = merged_df.drop_duplicates()

# Reset the index for easier manipulation
merged_df.reset_index(inplace=True)

#merged_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Visualize relationships between electricity prices and weather variables
#plt.figure(figsize=(10, 6))
#sns.lineplot(x='HourUTC', y='SpotPriceDKK', data=merged_df, label='DKK Price')
#sns.lineplot(x='HourUTC', y='tavg', data=merged_df, label='Avg Temp')
#plt.title('Electricity Spot Price and Average Temperature Over Time')
#plt.xlabel('Date')
#plt.ylabel('Values')
#plt.legend()
#plt.show()

# Explore correlations
#corr = merged_df[['SpotPriceDKK', 'SpotPriceEUR', 'precipitation', 'snow_depth', 'tavg', 'tmax', 'tmin']].corr()
#sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
#plt.title('Correlation Matrix')
#plt.show()


In [ ]:
# Define X (features) and y (target)
X = merged_df[['precipitation', 'snow_depth', 'tavg', 'tmax', 'tmin']]  # You can add more features
y = merged_df['SpotPriceDKK']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Mean Absolute Error: ", mean_absolute_error(y_test, y_pred))
